<a href="https://colab.research.google.com/github/Luly7/RT/blob/main/Roberta_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Luly7/RT.git

Cloning into 'RT'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 16 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (16/16), 4.64 MiB | 3.86 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [3]:
checkpoint_path = "/content/drive/MyDrive/checkpoints/ckpt-{epoch}"


This is TURBO COLAB TEMPLATE

In [10]:
# --- 0. Install everything ---
!pip install -q transformers tensorflow
!pip install tf-keras #added tf-keras

# --- 1. Imports ---
import tensorflow as tf
from tensorflow.keras import mixed_precision
from transformers import RobertaTokenizer, TFRobertaModel
import pandas as pd
from tqdm import tqdm

# --- 2. Enable Mixed Precision for Speed ---
mixed_precision.set_global_policy('mixed_float16')

# --- 3. Load your Data ---
csv_path = "/content/drive/MyDrive/CS6480/output_rt_fixed_smiles_unique.csv"  # << update if needed
df = pd.read_csv(csv_path)

# Confirm columns
print(df.columns)

# --- 4. Load Tokenizer and Model ---
model_name = "distilroberta-base"  # ⚡ smaller & faster (you can swap later)
tokenizer = RobertaTokenizer.from_pretrained(model_name)
transformer_model = TFRobertaModel.from_pretrained(model_name, from_pt=True, ignore_mismatched_sizes=True)

# Freeze Transformer
transformer_model.trainable = False

# --- 5. Define RT Prediction Model ---
class RTModel(tf.keras.Model):
    def __init__(self, transformer_model):
        super(RTModel, self).__init__()
        self.transformer = transformer_model
        self.dense = tf.keras.layers.Dense(1)

    def call(self, inputs):
        outputs = self.transformer(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            training=False
        ).last_hidden_state
        pooled_output = tf.reduce_mean(outputs, axis=1)
        rt_prediction = tf.cast(self.dense(pooled_output), dtype=tf.float32)  # fix dtype
        return rt_prediction

# Instantiate model
rt_model = RTModel(transformer_model)

# --- 6. Tokenize your SMILES ---
tokenized_inputs = tokenizer(
    list(df['fixed_smiles']),  # <<< Make sure your column is 'fixed_smiles'
    padding=True,
    truncation=True,
    max_length=128,  # << shorter inputs = faster
    return_tensors="tf"
)

# --- 7. Create TensorFlow Dataset ---
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': tokenized_inputs['input_ids'],
        'attention_mask': tokenized_inputs['attention_mask']
    },
    tf.constant(df['rt'].values, dtype=tf.float32)
))

# Shuffle, batch, prefetch
batch_size = 32  # <<< try increasing if you have GPU memory
train_dataset = dataset.shuffle(buffer_size=len(df)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# --- 8. Set Checkpoint Saving ---
checkpoint_path = "/content/drive/MyDrive/CS6480/checkpoints/ckpt-{epoch}.weights.h5"

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_freq='epoch',
    verbose=1
)

# --- 9. Compile the Model ---
rt_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
    loss='mse'
)

# --- 10. Train the Model ---
rt_model.fit(
    train_dataset,
    epochs=3,
    callbacks=[checkpoint_callback]
)

# --- 11. Save Final Model ---
rt_model.save('/content/drive/MyDrive/CS6480/rt_model_final.keras')
print("✅ Model training and saving complete!")


Index(['rt', 'fixed_smiles'], dtype='object')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/3
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 0s 620ms/step - loss: 634814.4375
Epoch 1: saving model to /content/drive/MyDrive/CS6480/checkpoints/ckpt-1.weights.h5
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 1557s 620ms/step - loss: 634801.1875
Epoch 2/3
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 0s 620ms/step - loss: 510238.4375
Epoch 2: saving model to /content/drive/MyDrive/CS6480/checkpoints/ckpt-2.weights.h5
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 1550s 620ms/step - loss: 510226.9688
Epoch 3/3
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 0s 625ms/step - loss: 402444.6875
Epoch 3: saving model to /content/drive/MyDrive/CS6480/checkpoints/ckpt-3.weights.h5
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 1563s 625ms/step - loss: 402434.6875
✅ Model training and saving complete!


In [ ]:
from transformers import RobertaTokenizer, TFRobertaModel

# 1. Rebuild model and tokenizer
model_name = "distilroberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
transformer_model = TFRobertaModel.from_pretrained(model_name, from_pt=True, ignore_mismatched_sizes=True)

# 2. Load your saved weights
checkpoint_path = "/content/drive/MyDrive/checkpoints/my_model.ckpt"
transformer_model.load_weights(checkpoint_path)

# 3. Now your model is restored and ready!
# You can continue training, or do predictions

# Example: predict something
# inputs = tokenizer("ClCCCN1CCCC1", return_tensors="tf")
# outputs = transformer_model(**inputs)


In [6]:
!ls -lh /content/checkpoints/


ls: cannot access '/content/checkpoints/': No such file or directory
